In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS

from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import PromptTemplate

from langchain.chains import RetrievalQA

In [3]:
loader = PyPDFDirectoryLoader("./dsa-patterns")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200)

final_docs = text_splitter.split_documents(docs)
len(final_docs)

62

In [5]:
# Embeddings using HuggingFace

hf_embeddings = HuggingFaceBgeEmbeddings(
    # Other option: sentence-transformers/all-MiniLM-16-v2 
    model_name = "BAAI/bge-small-en-v1.5", 
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings': True}

)


In [ ]:
import numpy as np
np.array(hf_embeddings.embed_query(final_docs[0].page_content))

In [7]:
vector_store = FAISS.from_documents(final_docs, hf_embeddings)


In [11]:
# Query using similarlity search
query = "whats the most important question from heaps"
relevant_docs = vector_store.similarity_search(query)

print(relevant_docs[0].page_content)

1.17 Top 5 Heap/Priority Queue Problems
Num Problem Summary
1 Meeting Rooms II LeetCode: Meeting Rooms II
2 Task Scheduler LeetCode: Task Scheduler
3 Last Stone Weight LeetCode: Last Stone Weight
4 The Skyline Problem LeetCode: The Skyline Problem
GitHub: https://github.com/dennyzhang/cheatsheet.dennyzhang.com/tree/master/cheatsheet-leetcode-A4 5 of 8


In [12]:
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k":3})
print(retriever)

tags=['FAISS', 'HuggingFaceBgeEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x177c959f0> search_kwargs={'k': 3}


In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['HUGGINGFACEHUB_API_TOKEN'] = os.getenv("HUGGINGFACEHUB_API_TOKEN")


In [26]:
from langchain_community.llms import HuggingFaceHub

hf = HuggingFaceHub(
    repo_id = "mistralai/Mistral-7B-v0.1",
    model_kwargs = {
        "temperature":0.1,
        "max_length": 500
        }
)

# Generic model response
query = "Whats most important question from heaps"
hf.invoke(query)

'Whats most important question from heaps of questions that I get asked is “How do I get started?”\n\nI’ve been asked this question so many times that I’ve decided to write a blog post about it.\n\nI’ve been asked this question so many times that I’ve decided to write a blog post about it.\n\nI’ve been asked this question so many times that I’ve decided to write a blog post about it.\n\nI’ve been asked this question so many'

In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

hf = HuggingFacePipeline.from_model_id(
    model_id = "mistralai/Mistral-7B-v0.1",
    task = "text-generation",
    pipeline_kwargs = {"temperature": 0,
                       "max_new_tokens": 300}
)

llm = hf
llm.invoke(query)

In [35]:
prompt_template = """

Use the following context to answer the question asked.
Please try to provide the answer only based on the context.

{context}
Question: {question}

Helpful answers:
"""

In [36]:
prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])

In [37]:
rqa = RetrievalQA.from_chain_type(
    llm = hf,
    chain_type = "stuff",
    retriever = retriever,
    return_source_documents = True,
    chain_type_kwargs = {"prompt": prompt}
)

In [38]:
query = "Whats the most important patterns"

In [39]:
result = rqa.invoke({"query": query})
print(result['result'])



Use the following context to answer the question asked.
Please try to provide the answer only based on the context.

5 
 
https://www.youtube.com/watch?v=OgX75yqQvC0 
LeetCode101: 20 Coding Patterns to Master 
MAANG Interviews 
Coding patterns enhance our “ability to map a new problem to an already known problem.” 
Here are the 20 coding patterns: 
1. Sliding Window 
2. Islands (Matrix Traversal) 
3. Two Pointers 
4. Fast & Slow Pointers 
5. Merge Intervals 
6. Cyclic Sort 
7. In-place Reversal of a LinkedList 
8. Tree Breadth-First Search 
9. Tree Depth First Search 
10. Two Heaps 
11. Subsets 
12. Modified Binary Search 
13. Bitwise XOR 
14. Top ‘K’ Elements 
15. K-way Merge 
16. Topological Sort 
17. Unbounded Knapsack 
18. Fibonacci Numbers 
19. Palindromic Subsequence 
20. Longest Common Substring

1 
 
https://designgurus.org/blog/top-lc-patterns 
Top LeetCode Patterns for FAANG Coding Interviews 
 
Preparing for coding interviews can be made easier by focusing on coding patter